In [67]:
#importing libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC


import time

import requests
from bs4 import BeautifulSoup
import pandas as pd


In [68]:
#scrape top 20 beginner-friendly anime
url = 'https://jw-webmagazine.com/anime-for-beginners/'

response = requests.get(url)

#put url into beautifulsoup
soup = BeautifulSoup(response.text, 'html.parser')

#get all 20 anime names 
anime_names = soup.find_all('h2', class_='')

#list for df
anime_list = []

#loop through the 20 animes 
for anime in anime_names:
    #get just the text
    anime_text = anime.text.strip()
    #split the anime text after the number and the dot and remove whitespaces from each part
    split_result = [part.strip() for part in anime_text.split('.')]
    #get the name of anime
    anime_name = split_result[1]
    #Bug Fix for 5. Cowboy Bepop
    if '&nbsp;' in anime_name:
        anime_name = anime_name.replace('&nbsp;', ' ')
    #append to list
    anime_list.append(anime_name)

#create DF 
df = pd.DataFrame(anime_list, columns=['Anime'])

#export df to csv. Added encoding to fix accent marks on animes
df.to_csv('top20_beginner_animes.csv', index=False, encoding='utf-8-sig')

print("Successfuly retrieved top 20 anime")
print("Exported as top20_beginner_animes.csv")



Successfuly retrieved top 20 anime
Exported as top20_beginner_animes.csv


In [ ]:
#Create a Chrome web driver and setting the options
chrome_options = Options()

#keep the browser window open until the user closes it
chrome_options.add_experimental_option("detach", True)

#instantiating the driver
driver = webdriver.Chrome(options= chrome_options)

#setting the url
url = "https://myanimelist.net/anime.php"
driver.get(url)

#keywords to enter to search bar
print(anime_list)

#lists to store data
anime_score_list = []
anime_rank_list = []
anime_popularity_list = []
anime_total_watched_list = []
anime_episodes_list = []

for anime in anime_list:
    driver.implicitly_wait(2)
    time.sleep(2)
    #select search box 
    search = wait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'q')))
    print(anime)
    
    driver.implicitly_wait(2)
    search.clear()

    #clear the input field before sending keys
    driver.execute_script("arguments[0].value = '';", search)
    
    
    
    #enter anime name to search box
    search.send_keys(anime)

    driver.implicitly_wait(2)

    #select the search button
    search_button = wait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'inputButton')))

    #click button
    search_button.click()

    #select anime link a tag
    anime_link = wait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'hoverinfo_trigger')))
    #click anime link to its page
    anime_link.click()

    #anime score
    anime_score = driver.find_element(By.XPATH, '//*[@id="content"]/table/tbody/tr/td[2]/div[1]/table/tbody/tr[1]/td/div[1]/div[1]/div[1]/div[1]/div[1]/div').text
    anime_score = float(anime_score)
    print("anime score")
    print(anime_score)


    #anime rank
    anime_rank = driver.find_element(By.XPATH, '//*[@id="content"]/table/tbody/tr/td[2]/div[1]/table/tbody/tr[1]/td/div[1]/div[1]/div[1]/div[1]/div[2]/span[1]/strong').text
    anime_rank = anime_rank.replace('#','')
    anime_rank = float(anime_rank)
    print("rank")
    print(anime_rank)
    

    #anime popularity
    anime_popularity = driver.find_element(By.XPATH, '//*[@id="content"]/table/tbody/tr/td[2]/div[1]/table/tbody/tr[1]/td/div[1]/div[1]/div[1]/div[1]/div[2]/span[2]/strong').text
    anime_popularity = anime_popularity.replace('#', '')
    anime_popularity = float(anime_popularity)
    print("popularity")
    print(anime_popularity)
    
    #anime_total_watched (members)
    anime_total_watched = driver.find_element(By.XPATH,'//*[@id="content"]/table/tbody/tr/td[2]/div[1]/table/tbody/tr[1]/td/div[1]/div[1]/div[1]/div[1]/div[2]/span[3]/strong').text
    anime_total_watched = anime_total_watched.replace(',', '')
    anime_total_watched = float(anime_total_watched)
    print("members")
    print(anime_total_watched)

    #anime total episodes
    #anime_episodes = driver.find_element(By.XPATH, '//*[@id="content"]/table/tbody/tr/td[1]/div/div[10]').text
    #anime_episodes = anime_episodes.replace('Episodes: ', '')
    #driver.implicitly_wait(2)
    #anime_episodes = float(anime_episodes)
    #print(anime_episodes)

    divs = driver.find_elements(By.XPATH, '//div[@class="spaceit_pad"]')
    # iterate over divs and find the one with text that starts with "Episodes:"
    for div in divs:
        # get the text in div
        text = div.text  
        # check if the text starts with 'Episodes:'
        if text.startswith('Episodes:'):  
            # get the number of episodes
            anime_episodes = text.replace('Episodes: ', '')
            anime_episodes = float(anime_episodes)
            print("episodes")
            print(anime_episodes)
            # break the loop once the div is found
            break  


    anime_score_list.append(anime_score)
    anime_rank_list.append(anime_rank)
    anime_popularity_list.append(anime_popularity)
    anime_total_watched_list.append(anime_total_watched)
    anime_episodes_list.append(anime_episodes)

    #return to search 
    driver.get(url)





In [123]:
df = pd.DataFrame({
                    'score': anime_score_list,
                    'Rank': anime_rank_list,
                    'Popularity': anime_popularity_list,
                    'Members': anime_total_watched_list,
                    'Episodes': anime_episodes_list
})

print(df)

    score    Rank  Popularity    Members  Episodes
0    7.50  1713.0        57.0  1578358.0      12.0
1    8.26   280.0        15.0  2332533.0     500.0
2    9.10     1.0         3.0  3155547.0      64.0
3    8.53   110.0         1.0  3720820.0      25.0
4    8.38   192.0       601.0   358854.0       1.0
5    7.96   640.0       149.0   983161.0     153.0
6    7.37  2179.0       326.0   604522.0     276.0
7    6.70  5330.0        35.0  1893158.0      24.0
8    7.09  3575.0      1663.0   121917.0       1.0
9    8.64    69.0        95.0  1246092.0      12.0
10   8.90    23.0       371.0   539582.0       1.0
11   8.63    72.0        89.0  1323511.0      12.0
12   8.08   484.0        16.0  2294579.0      12.0
13   7.61  1357.0      1088.0   204049.0       1.0
14   8.51   124.0       106.0  1180523.0      26.0
15   8.51   123.0         7.0  2778724.0      26.0
16   7.90   724.0       187.0   845443.0      12.0
17   8.31   239.0        30.0  1981143.0      12.0
18   7.89   742.0         6.0  

In [124]:
df.describe()

,score,Rank,Popularity,Members,Episodes
count,20.00000,20.000000,20.000000,2.000000e+01,20.000000
mean,8.11050,909.500000,244.350000,1.588915e+06,60.350000
std,0.61363,1379.530984,426.328175,1.033673e+06,122.475465
min,6.70000,1.000000,1.000000,1.219170e+05,1.000000
25%,7.82000,119.750000,15.750000,7.852128e+05,12.000000
50%,8.28500,259.500000,73.000000,1.450934e+06,12.500000
75%,8.51500,895.750000,221.750000,2.304068e+06,26.000000
max,9.10000,5330.000000,1663.000000,3.720820e+06,500.000000
